In [1]:
##### Importing all the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
#Join the 8 csv files into a single dataframe
# Define the folder path containing the CSV files
folder_path = r'c:\Users\gunit\Desktop\Anomaly_Detection\CICIDS_2017'

# List all files in the folder and filter for .csv files
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read and concatenate all CSV files into a single DataFrame
df = pd.concat(
    [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files],
    ignore_index=True
)

df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
del csv_files,folder_path

In [5]:
# Strip leading and trailing spaces from column names
df.columns = df.columns.str.strip()
df.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [6]:
df['Label'].value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [7]:
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [8]:
#clean the values in label column
df['Label']=df['Label'].str.replace('�','-')
df['Label'].unique()                                

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack - Brute Force', 'Web Attack - XSS',
       'Web Attack - Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [9]:
# Drop rows with null values ,infinity or -infinity values in any column
df=df.dropna()
df = df.replace([np.inf, -np.inf], np.nan).dropna()

In [10]:
print(df['Fwd Header Length.1'].describe())
print(df['Fwd Header Length'].describe())

count    2.827876e+06
mean    -2.602379e+04
std      2.106353e+07
min     -3.221223e+10
25%      4.000000e+01
50%      6.400000e+01
75%      1.200000e+02
max      4.644908e+06
Name: Fwd Header Length.1, dtype: float64
count    2.827876e+06
mean    -2.602379e+04
std      2.106353e+07
min     -3.221223e+10
25%      4.000000e+01
50%      6.400000e+01
75%      1.200000e+02
max      4.644908e+06
Name: Fwd Header Length, dtype: float64


In [11]:
df=df.drop('Fwd Header Length.1',axis=1)

In [12]:
# Function to map ports to categories
def port_range(port):
    if port <= 1023:
        return 'well_known'
    elif port <= 49151:
        return 'registered'
    else:
        return 'dynamic'

# Apply grouping
df['Destination_port_group'] = df['Destination Port'].apply(port_range)
df['Destination_port_group'] = df['Destination_port_group'].astype('category')  

df=df.drop('Destination Port',axis=1)

In [13]:
negative_counts = (df.select_dtypes(include='number') < 0).sum()
negative_columns = negative_counts[negative_counts > 0]

print("Columns with negative values and their counts:")
print(negative_columns)

Columns with negative values and their counts:
Flow Duration                  115
Flow Bytes/s                    85
Flow Packets/s                 115
Flow IAT Mean                  115
Flow IAT Max                   115
Flow IAT Min                  2890
Fwd IAT Min                     17
Fwd Header Length               35
Bwd Header Length               22
Init_Win_bytes_forward     1001172
Init_Win_bytes_backward    1439672
min_seg_size_forward            35
dtype: int64


In [14]:
#Remove rows with negative valus in header length column
df = df[(df['Fwd Header Length'] >= 0) & (df['Bwd Header Length'] >= 0)]

In [16]:
#Replace negative values with '0' 
for col in negative_columns.index:
    df[col] = df[col].apply(lambda x: 0 if x < 0 else x)   

In [18]:
#rechecking for negative values
negative_counts = (df.select_dtypes(include='number') < 0).sum()
negative_columns = negative_counts[negative_counts > 0]

print("Checking for columns after replacement:")
print(negative_columns)

Checking for columns after replacement:
Series([], dtype: int64)


In [19]:
#convert flag columns and destination port into categorical columns 
cols_to_convert=['Fwd PSH Flags', 'Bwd PSH Flags','Fwd URG Flags', 'Bwd URG Flags','FIN Flag Count','SYN Flag Count','RST Flag Count','PSH Flag Count','ACK Flag Count','URG Flag Count','CWE Flag Count','ECE Flag Count','Label']
df[cols_to_convert] = df[cols_to_convert].astype('category')

In [20]:
del cols_to_convert

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2827841 entries, 0 to 2830742
Data columns (total 78 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   Flow Duration                int64   
 1   Total Fwd Packets            int64   
 2   Total Backward Packets       int64   
 3   Total Length of Fwd Packets  int64   
 4   Total Length of Bwd Packets  int64   
 5   Fwd Packet Length Max        int64   
 6   Fwd Packet Length Min        int64   
 7   Fwd Packet Length Mean       float64 
 8   Fwd Packet Length Std        float64 
 9   Bwd Packet Length Max        int64   
 10  Bwd Packet Length Min        int64   
 11  Bwd Packet Length Mean       float64 
 12  Bwd Packet Length Std        float64 
 13  Flow Bytes/s                 float64 
 14  Flow Packets/s               float64 
 15  Flow IAT Mean                float64 
 16  Flow IAT Std                 float64 
 17  Flow IAT Max                 int64   
 18  Flow IAT Min               

In [22]:
df['Label'].value_counts()

Label
BENIGN                        2271285
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack - Brute Force         1507
Web Attack - XSS                  652
Infiltration                       36
Web Attack - Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [24]:
save_path = r'c:\Users\gunit\Desktop\Anomaly_Detection\clean_data.csv'
df.to_csv(save_path, index=False)